In [25]:
def go_home(final):
    homeposition = ['home',0,0]
    distance = ((final[1] - homeposition[1]) ** 2 + (final[2] - homeposition[2]) ** 2) ** 0.5
    return distance

def manipulate_orders(orders):
    my_list = []
    for i in range(len(orders)):
        my_list.append([orders[i][0], float(orders[i][1]), float(orders[i][2])])
    return my_list

def find_shortest_path(orders_list):
    shortest_distance = {}
    for i in range(len(orders_list)):
        if (orders_list[i][0] != 'home'):
            shortest_distance[i] = ((orders_list[0][1] - orders_list[i][1]) ** 2 + (orders_list[0][2] - orders_list[i][2]) ** 2) ** 0.5
    return [min(shortest_distance, key=shortest_distance.get), shortest_distance[min(shortest_distance, key=shortest_distance.get)]]

def remove_from_list(string, orders_list):
    for orders in orders_list:
        if string in orders:
            orders_list.remove(orders)
            return orders_list
        
def create_tmp_list(string, orders_list):
    for orders in orders_list:
        if string in orders:
            return orders
        
def find_next_shortest_path(my_list, orders_list):
    shortest_distance = {}
    for i in range(len(orders_list)):
        shortest_distance[orders_list[i][0]] = ((my_list[1] - orders_list[i][1]) ** 2 + (my_list[2] - orders_list[i][2]) ** 2) ** 0.5
    return [min(shortest_distance, key=shortest_distance.get), shortest_distance[min(shortest_distance, key=shortest_distance.get)]]

def find_next_shortest_paths(my_list, orders_list):
    shortest_distance = {}
    for i in range(len(orders_list)):
        shortest_distance[orders_list[i][0]] = ((my_list[1] - orders_list[i][1]) ** 2 + (my_list[2] - orders_list[i][2]) ** 2) ** 0.5
    sorted_dict = [[k, shortest_distance[k]] for k in sorted(shortest_distance, key=shortest_distance.get)]
    return sorted_dict


def check_next_shortest_path(my_list, orders_list):
    shortest_distance = {}
    for i in range(len(orders_list)):
        if (my_list[0] != orders_list[i][0]):
            shortest_distance[orders_list[i][0]] = ((my_list[1] - orders_list[i][1]) ** 2 + (my_list[2] - orders_list[i][2]) ** 2) ** 0.5
    return [min(shortest_distance, key=shortest_distance.get), shortest_distance[min(shortest_distance, key=shortest_distance.get)]]

def compare_distance(my_list, orders_list):
    distance = 0
    tmp_distance = 0
    list_to_return = False
    for i in range(len(my_list)):
        for order in orders_list:
            if my_list[i][0] in order:
                if (len(orders_list) == 1):
                    return [my_list[i][0], my_list[i][1]]
                else:
                    next_shortest = check_next_shortest_path(order, orders_list)
                if distance == 0:  
                    distance = my_list[i][1] + next_shortest[1]
                    list_to_return = [my_list[i][0], next_shortest[0]]
                else:
                    tmp_distance = my_list[i][1] + next_shortest[1]
                    if distance > tmp_distance:
                        list_to_return = []
                        distance = tmp_distance
                        list_to_return = [my_list[i][0], next_shortest[0]]
                break
                
    return [list_to_return, distance]
                        
def two_opt(route, orders_list):
    num_routes = len(route)
    for k in range(num_routes):
        for i in range(num_routes - 2):
            for j in range(i + 2, num_routes - 1):
                change = dist(route[i], route[j], orders_list) + dist(route[i+1], route[j+1], orders_list) - dist(route[i], route[i+1], orders_list) - dist(route[j], route[j+1], orders_list)
                if change < 0:
                    min_i, min_j = i, j
                    route[min_i+1:min_j+1] = route[min_i+1:min_j+1][::-1]
    return route
    
def dist(a, b, orders_list):
    my_list1 = []
    my_list2 = []
    if a == 'home':
        my_list1 = ['home', 0, 0]
    for orders in orders_list:
        if a in orders:
            my_list1 = orders
        elif b in orders:
            my_list2 = orders
    return ((my_list1[1] - my_list2[1]) ** 2 + (my_list1[2] - my_list2[2]) ** 2) ** 0.5

In [2]:
# NEAREST NEIGHBOURS
from utility import *

check_milage = []
counter = 0
totalmilage = 0
orders_to_return = []
tmp_list = []
orders = list_reader('orders.csv')
parameters = parameter_reader('parameters.csv')
orders_list = manipulate_orders(orders)
tmp = []
number_trucks = 25
threshold = len(orders) // number_trucks
threshold += 1

shortest_path_from_home = find_shortest_path(orders_list)

if (shortest_path_from_home[0] < 10):
    string_to_return = "O0000" + str(shortest_path_from_home[0])
elif (shortest_path_from_home[0] < 100):
    string_to_return = "O000" + str(shortest_path_from_home[0])
elif (shortest_path_from_home[0] < 1000):
    string_to_return = "O00" + str(shortest_path_from_home[0])

tmp_list.append(string_to_return)

totalmilage = shortest_path_from_home[1]
tmp = create_tmp_list(string_to_return, orders_list)
orders_list = remove_from_list(string_to_return, orders_list)
counter = counter + 1

while (len(orders_list) > 0): 
    if (counter < threshold):
        tmp_shortest = find_next_shortest_path(tmp, orders_list)
        tmp_list.append(tmp_shortest[0])
        totalmilage = totalmilage + tmp_shortest[1]
        tmp = create_tmp_list(tmp_shortest[0], orders_list)
        orders_list = remove_from_list(tmp_shortest[0], orders_list)
        counter = counter + 1
    else:
        orders_to_return.append(tmp_list)
        counter = 0
        tmp_list = []
        check_milage.append(totalmilage)
        totalmilage = 0

distance_from_home = go_home(tmp)
totalmilage += distance_from_home
orders_to_return.append(tmp_list)
check_milage.append(totalmilage)
number_of_trucks = len(orders_to_return)
orders_to_return[0].insert(0,'home')

orders_list = manipulate_orders(orders)

final_orders = []

for check_orders in orders_to_return:
    my_orders = two_opt(check_orders, orders_list)
    final_orders.append(my_orders)

final_orders[0].pop(0)

print(final_orders)


[['O00165', 'O00279', 'O00205', 'O00219', 'O00052', 'O00146', 'O00305', 'O00302', 'O00000', 'O00176', 'O00098', 'O00265', 'O00170'], ['O00070', 'O00185', 'O00062', 'O00152', 'O00042', 'O00036', 'O00311', 'O00055', 'O00169', 'O00208', 'O00011', 'O00003', 'O00237'], ['O00102', 'O00048', 'O00027', 'O00193', 'O00147', 'O00135', 'O00113', 'O00155', 'O00079', 'O00253', 'O00263', 'O00050', 'O00030'], ['O00004', 'O00199', 'O00132', 'O00294', 'O00112', 'O00184', 'O00204', 'O00013', 'O00231', 'O00245', 'O00122', 'O00087', 'O00240'], ['O00269', 'O00143', 'O00156', 'O00175', 'O00271', 'O00078', 'O00092', 'O00126', 'O00107', 'O00129', 'O00117', 'O00217', 'O00153'], ['O00239', 'O00018', 'O00283', 'O00238', 'O00233', 'O00137', 'O00298', 'O00059', 'O00179', 'O00065', 'O00124', 'O00026', 'O00086'], ['O00045', 'O00020', 'O00076', 'O00229', 'O00259', 'O00236', 'O00012', 'O00218', 'O00257', 'O00196', 'O00177', 'O00007', 'O00071'], ['O00254', 'O00174', 'O00194', 'O00021', 'O00080', 'O00084', 'O00116', 'O00

In [11]:
#ANT COLONY

from utility import *

counter = 0
totalmilage = 0
orders_to_return = []
tmp_list = []
orders = list_reader('orders.csv')
parameters = parameter_reader('parameters.csv')
orders_list = manipulate_orders(orders)
tmp = []

shortest_path_from_home = find_shortest_paths(orders_list)
check = compare_distance(shortest_path_from_home, orders_list)
totalmilage = check[1]
tmp = create_tmp_list(check[0][1], orders_list)
tmp_list.append(check[0][0])
orders_list = remove_from_list(check[0][0], orders_list)
counter = counter + 1


while (len(orders_list) > 0):
#     if (counter < 13):
        tmp_shortest = find_next_shortest_paths(tmp, orders_list)
        check = compare_distance(tmp_shortest, orders_list)
        if (len(orders_list) == 1):
            tmp_list.append(check[0])
            totalmilage += check[1]
            orders_list = remove_from_list(check[0], orders_list)
            counter = counter + 1
#         else:
#             totalmilage += check[1]
#             tmp = create_tmp_list(check[0][1], orders_list)
#             tmp_list.append(check[0][0])
#             orders_list = remove_from_list(check[0][0], orders_list)
#             counter = counter + 1
    else:
        orders_to_return.append(tmp_list)
        counter = 0
        tmp_list = []
        
orders_to_return.append(tmp_list)






 


KeyboardInterrupt: 

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale

import sklearn.metrics as sm
from sklearn import datasets
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
# Calculate the euclidian distance in n-space of the route r traversing cities c, ending at the path start.
path_distance = lambda r,c: np.sum([np.linalg.norm(c[r[p]]-c[r[p-1]]) for p in range(len(r))])
# Reverse the order of all elements from element i to element k in array r.
two_opt_swap = lambda r,i,k: np.concatenate((r[0:i],r[k:-len(r)+i-1:-1],r[k+1:len(r)]))

def two_opt(cities,improvement_threshold): # 2-opt Algorithm adapted from https://en.wikipedia.org/wiki/2-opt
    route = np.arange(cities.shape[0]) # Make an array of row numbers corresponding to cities.
    improvement_factor = 1 # Initialize the improvement factor.
    best_distance = path_distance(route,cities) # Calculate the distance of the initial path.
    while improvement_factor > improvement_threshold: # If the route is still improving, keep going!
        distance_to_beat = best_distance # Record the distance at the beginning of the loop.
        for swap_first in range(1,len(route)-2): # From each city except the first and last,
            for swap_last in range(swap_first+1,len(route)): # to each of the cities following,
                new_route = two_opt_swap(route,swap_first,swap_last) # try reversing the order of these cities
                new_distance = path_distance(new_route,cities) # and check the total distance with this modification.
                if new_distance < best_distance: # If the path distance is an improvement,
                    route = new_route # make this the accepted best route
                    best_distance = new_distance # and update the distance corresponding to this route.
        improvement_factor = 1 - best_distance/distance_to_beat # Calculate how much the route has improved.
    return route # When the route is no longer improving substantially, stop searchi

In [ ]:
from utility import *
orders = list_reader('orders.csv')
orders_list = manipulate_orders(orders)
orders_list = np.asarray(orders_list)
route = two_opt(orders_list,0.001)


In [ ]:
def total_length( nodes, solution ):
    objective = distance( solution[-1], solution[0])
    for i in range(0, len(solution)-3):
        for j in range(i+2, len(solution)-1):
        # evaluate 2-opt on edge (solution[i],solution[i+1]) and 
        # edge (solution[j],solution[j+1])




In [ ]:
for i in range(11):
    for j in range(i+2, 12):
        print(j)
        print(j+1)
        print(j+2)
        print('-----------------------')

In [18]:
def manipulate_orders(orders):
    my_list = []
    for i in range(len(orders)):
        my_list.append([orders[i][0], float(orders[i][1]), float(orders[i][2])])
    return my_list

def find_shortest_path(orders_list):
    shortest_distance = {}
    homeposition = ['home',0,0]
    for i in range(len(orders_list)):
        shortest_distance[i] = ((homeposition[1] - orders_list[i][1]) ** 2 + (homeposition[2] - orders_list[i][2]) ** 2) ** 0.5
    return [max(shortest_distance, key=shortest_distance.get), shortest_distance[max(shortest_distance, key=shortest_distance.get)]]

def remove_from_list(string, orders_list):
    for orders in orders_list:
        if string in orders:
            orders_list.remove(orders)
            return orders_list

def create_tmp_list(string, orders_list):
    for orders in orders_list:
        if string in orders:
            return orders

def find_next_shortest_path(my_list, orders_list):
    shortest_distance = {}
    for i in range(len(orders_list)):
        shortest_distance[orders_list[i][0]] = ((my_list[1] - orders_list[i][1]) ** 2 + (my_list[2] - orders_list[i][2]) ** 2) ** 0.5
    return [min(shortest_distance, key=shortest_distance.get),shortest_distance[min(shortest_distance, key=shortest_distance.get)]]  

def two_opt(route, orders):
    num_routes = len(route)
    for i in range(num_routes - 2):
        for j in range(i + 2, num_routes - 1):
            change0 = dist(route[i], route[j], orders) + dist(route[i + 1], route[j + 1], orders) - dist(route[i],route[i + 1],orders) - dist(route[j], route[j + 1], orders)
            change1 = dist(route[j], route[i+1], orders) + dist(route[i], route[j + 1], orders) - dist(route[i], route[j], orders) + dist(route[i + 1], route[j + 1], orders)
            change2 = dist(route[i+1], route[j], orders) + dist(route[i], route[j + 1], orders) - dist(route[j], route[i+1], orders) + dist(route[i], route[j + 1], orders)
            if change0 < 0:
                min_i, min_j = i, j
                if change1 < change0:
                    route[min_i + 1:min_j + 1] = route[min_i + 1:min_j + 1][::-1]
                elif change2 < change0:
                    route[min_i:min_j] = route[min_i:min_j][::-1]
                    route[min_i + 1:min_j + 1] = route[min_i + 1:min_j + 1][::-1]
                else:
                    route[min_i + 1:min_j + 1] = route[min_i + 1:min_j + 1][::-1]
    return route

def dist(a, b, orders_list):
    my_list1 = []
    my_list2 = []
    if a == 'home':
        my_list1 = ['home', 0, 0]
    elif b == 'home':
        my_list2 = ['home',0,0]
    for orders in orders_list:
        if a in orders:
            my_list1 = orders
        elif b in orders:
            my_list2 = orders
    return ((my_list1[1] - my_list2[1]) ** 2 + (my_list1[2] - my_list2[2]) ** 2) ** 0.5

# replace the content of this function with your own algorithm
def schedule_q1(orders, number_trucks):

    totalmilage = 0
    orders_to_return = []
    tmp_list = []
    orders_list = manipulate_orders(orders)
    tmp = []
    threshold = len(orders) // number_trucks
    threshold += 1

    shortest_path_from_home = find_shortest_path(orders_list)

    if (shortest_path_from_home[0] < 10):
        string_to_return = "O0000" + str(shortest_path_from_home[0])
    elif (shortest_path_from_home[0] < 100):
        string_to_return = "O000" + str(shortest_path_from_home[0])
    elif (shortest_path_from_home[0] < 1000):
        string_to_return = "O00" + str(shortest_path_from_home[0])

    tmp_list.append(string_to_return)
        
    tmp = create_tmp_list(string_to_return, orders_list)
    orders_list = remove_from_list(string_to_return, orders_list)

    while (len(orders_list) > 0):
            tmp_shortest = find_next_shortest_path(tmp, orders_list)
            tmp_list.append(tmp_shortest[0])
            tmp = create_tmp_list(tmp_shortest[0], orders_list)
            orders_list = remove_from_list(tmp_shortest[0], orders_list)
            



    
    final_orders = []
    complete_order = []
    tmp_final_order = []
    orders_list = manipulate_orders(orders)

    
    final_orders = two_opt(tmp_list, orders_list)

    for order in final_orders:
        if len(tmp_final_order) == threshold:
            complete_order.append(tmp_final_order)
            tmp_final_order = []
            tmp_final_order.append(order)
        else:
            tmp_final_order.append(order)
            
    complete_order.append(tmp_final_order)

    

    return complete_order

   



In [19]:
from utility import *
import copy

def legality_checking(truck_paths, order_list, number_trucks):
	check_trucks(truck_paths, number_trucks)
	checking_all_order(truck_paths, order_list)
	if checking_if_airports(truck_paths):
		print("no airport in q1")
		exit()

# replace these parameters with different csv file locations
order_csv = "./dataset/orders_001.csv"
parameter_csv = "./dataset/parameters_001.csv"

number_trucks, truck_speed, plane_speed = parameter_reader(parameter_csv)
orders = list_reader(order_csv)
location_dict = location_dict_generation([orders])

# orders_clone will be used on line 26. just in case your schedule_q1 on line 24 modifies orders
orders_clone = copy.deepcopy(orders) # <--- changed 

# call your function
truck_paths = schedule_q1(orders, number_trucks)  

legality_checking(truck_paths, orders_clone, number_trucks)  # <--- changed

print("Score for q1 is:", scoring_q1(truck_paths, location_dict)) # print your score. the smaller the better













Score for q1 is: 144.13541466344313


In [43]:
a = [1,2,3,4,5]

a.insert(2, 'cat')
print(a)

[1, 2, 'cat', 3, 4, 5]


[2, 1, 0, 3]
